In [1]:
K.<a> = NumberField(x - 1)
from sage.rings.polynomial.cyclotomic import cyclotomic_coeffs
S = ZZ['x']
K.<w> = NumberField(x^2 - x + 1)
R.<x> = PolynomialRing(ZZ)



def is_relatively_prime(m,n):
    if gcd(m,n) ==1:
        return 1
    else:
        return 0 
    
def raw_fekete(n):
    "Fekete polynomial with Delta odd "
    D=n
    v=[is_relatively_prime(a+1,n) for a in range(0,D-1)]
    F_D=R(v)
    return F_D*x  

def fekete_by_last_factor(n):
    F=raw_fekete(n)
    f = F.factor()[-1][0]
    return f

def divisor_set_general(q,p):
    divisor = set()
    for d in range(2, q):
        if q%d ==1:
            divisor.add(d)
    for d in range(2,p):
        if p%d==1 and d !=q:
            divisor.add(d)
    for d in range(2,p+q+1):
        if (q*p+1)%d==0 and (p+q)%d==0:
            divisor.add(d)
    return divisor 
   
    
def cyclotomic_factor(q,p):
    """
    return the cyclotmic factor of F_{qp}
    """
    cyc_factor =x
    divisor_set = divisor_set_general(q,p)
    for d in divisor_set:
        cyc_factor *=S(cyclotomic_coeffs(d))
    return cyc_factor


def fekete(q,p):
    n=p*q
    cyc_factor = cyclotomic_factor(q,p)
    F= raw_fekete(n)
    f,r =F.quo_rem(cyc_factor) 
    return f 


def reduced_fekete(f):
    u=f.trace_polynomial()
    g_D=u[0]
    return g_D

def fekete_reduction(f, q):
    f=f.change_ring(GF(q))
    return f.factor()        


def two_four_cycle(f,q):
    factor = fekete_reduction(f,q)
    number_of_factor = len(factor)
    #check that f is separable modulo q
    for i in range(number_of_factor):
        if factor[i][1] >1:
            return False
    number_two_cycle =0 
    number_four_cycle =0 
    number_even_cycle =0
    for i in range(number_of_factor):
        if factor[i][0].degree() ==2:
            number_two_cycle +=1
        if factor[i][0].degree() == 4:
            number_four_cycle +=1 
        if factor[i][0].degree() % 2 ==0:
            number_even_cycle +=1
    if number_two_cycle ==1 and number_four_cycle ==1 and number_even_cycle ==2:
        return True
    return False
            
    
def search_two_four_cycle(f,n):
    for q in range(n):
        if is_prime(q):
            if two_four_cycle(f,q):
                return q
    return -1 

def search_two_four_cycle_2(f,m,n):
    for q in range(m,n):
        if is_prime(q):
            if two_four_cycle(f,q):
                return q
    return -1 



def almost_cycle(f,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(f,q)
            if len(factor)==3: 
                factor1=factor[0][0]
                factor2=factor[1][0]
                degree1=factor1.degree()
                degree2=factor2.degree()
                if degree1==1 and degree2==1 and factor[0][1]==1 and factor[1][1]==1 and factor[2][1]==1: 
                    return q
    return  -1         
                
def irreducible(f,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(f,q)
            if len(factor)==1 and factor[0][1]==1:
                    return q
    return  -1         
                
       
    
def length_test_2(v):
    #count the number of even entries in v
    count2=0
    for item in v:
        if item==2:
            count2 +=1
    count_even=0     
    for item in v:        
        if item %2 ==0:
            count_even +=1
    if count2==count_even==1:
        return True
    return False    

def length_test_4(v):
    #count the number of even entries in v
    count4=0
    for item in v:
        if item==4:
            count4 +=1
    count_even=0     
    for item in v:        
        if item %2 ==0:
            count_even +=1
    if count4==count_even==1:
        return True
    return False    

    
def two_cycle(f,n):
    result=[]
    for q in range(n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_2(v):
            return q
    return -1    

def two_cycle_2(f,m,n):
    result=[]
    for q in range(m,n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_2(v):
            return q
    return -1 

def four_cycle(f,n):
    result=[]
    for q in range(n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_4(v):
            return q
    return -1  

def cycle(g,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(g,q)
            if len(factor)==2: 
                factor1=factor[0][0]
                coef=factor1.degree()
                if coef==1 and factor[0][1]==1 and factor[1][1]==1: 
                                   return q
    return  -1   
                    

                    
def search_quadruple(f,n):
    irr=irreducible(f,n)
    print(f"f is irreducible at q= ", irr)
    q_cycle=almost_cycle(f,n)
    print(f"f has an (2m-2) cycle at q=", q_cycle)
    q_tranposition=two_cycle(f,n)
    print(f"f has an 2-cycle at q=", q_tranposition)
    q_four_cycle=four_cycle(f,n)
    print(f"f has an 4-cycle at q=", q_four_cycle)
    

def quadruple(f,n):
    irr=irreducible(f,n)
    q_cycle=almost_cycle(f,n)
    q_tranposition=two_cycle(f,n)
    q_four_cycle=four_cycle(f,n)
    result=(irr, q_cycle, q_tranposition, q_four_cycle)
    return result
    
    
def triple(g,n):
    irr=irreducible(g,n)
    q_cycle=cycle(g,n)
    q_tranposition=two_cycle(g,n)
    result=[irr, q_cycle, q_tranposition]
    return result

As usual, we test the above code with a concrete example.

In [8]:
q=7
p=11
n=q*p
F=raw_fekete(n)
print(F)

x^76 + x^75 + x^74 + x^73 + x^72 + x^71 + x^69 + x^68 + x^67 + x^65 + x^64 + x^62 + x^61 + x^60 + x^59 + x^58 + x^57 + x^54 + x^53 + x^52 + x^51 + x^50 + x^48 + x^47 + x^46 + x^45 + x^43 + x^41 + x^40 + x^39 + x^38 + x^37 + x^36 + x^34 + x^32 + x^31 + x^30 + x^29 + x^27 + x^26 + x^25 + x^24 + x^23 + x^20 + x^19 + x^18 + x^17 + x^16 + x^15 + x^13 + x^12 + x^10 + x^9 + x^8 + x^6 + x^5 + x^4 + x^3 + x^2 + x


In [9]:
f=fekete(q,p)
print(f)

x^62 - x^60 + x^55 - x^53 + x^51 - x^49 + x^48 - 2*x^46 + 2*x^45 + x^44 - 2*x^43 + x^41 + x^38 - x^37 - 2*x^36 + x^35 + 2*x^34 - x^32 - x^30 + 2*x^28 + x^27 - 2*x^26 - x^25 + x^24 + x^21 - 2*x^19 + x^18 + 2*x^17 - 2*x^16 + x^14 - x^13 + x^11 - x^9 + x^7 - x^2 + 1


In [10]:
g=reduced_fekete(f)
print(g)

x^31 - 32*x^29 + 463*x^27 - 4004*x^25 + x^24 + 23050*x^23 - 25*x^22 - 93104*x^21 + 275*x^20 + 270963*x^19 - 1750*x^18 - 574331*x^17 + 7124*x^16 + 885647*x^15 - 19361*x^14 - 981770*x^13 + 35551*x^12 + 763583*x^11 - 43579*x^10 - 399786*x^9 + 34277*x^8 + 131307*x^7 - 15907*x^6 - 23669*x^5 + 3646*x^4 + 1706*x^3 - 264*x^2 - 24*x + 2


In [11]:
fekete_reduction(g,19)

x^31 + 6*x^29 + 7*x^27 + 5*x^25 + x^24 + 3*x^23 + 13*x^22 + 15*x^21 + 9*x^20 + 4*x^19 + 17*x^18 + x^17 + 18*x^16 + 17*x^13 + 2*x^12 + 11*x^11 + 7*x^10 + 12*x^9 + x^8 + 17*x^7 + 15*x^6 + 5*x^5 + 17*x^4 + 15*x^3 + 2*x^2 + 14*x + 2

In [12]:
fekete_reduction(g,139)

(x + 80) * (x^30 + 59*x^29 + 113*x^28 + 134*x^27 + 29*x^26 + 43*x^25 + 62*x^24 + 45*x^23 + 129*x^22 + 80*x^21 + 20*x^20 + 65*x^19 + 134*x^18 + 40*x^17 + 14*x^16 + 27*x^15 + 3*x^14 + 137*x^13 + 8*x^12 + 22*x^11 + 103*x^10 + 28*x^9 + 101*x^8 + 65*x^7 + 34*x^6 + 138*x^5 + 41*x^4 + 88*x^3 + 87*x^2 + 4*x + 73)

In [13]:
fekete_reduction(g,13)

(x^2 + 7*x + 3) * (x^3 + 11*x^2 + 5*x + 10) * (x^7 + 3*x^5 + 2*x^4 + 9*x^3 + 6*x^2 + 4*x + 10) * (x^19 + 8*x^18 + 9*x^17 + x^16 + 3*x^15 + x^14 + 12*x^13 + 3*x^12 + 10*x^11 + 9*x^10 + 5*x^9 + 6*x^8 + x^7 + 10*x^6 + 10*x^5 + 12*x^4 + 4*x^2 + 12*x + 2)

# Galois group of $g_{7p}$

In [5]:
P=Primes()
n=10**5
q=7
p=q
while p < 200: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

11 [19, 139, 13]
13 [17, 107, 43]


17 [373, 41, 29]


19 [53, 401, 43]


23 [113, 397, 19]


29 [1493, 421, 19]


...

191 [1123, 79, 13]


193 [12011, 613, 79]


197 [757, 1753, 661]


199 [6871, 919, 701]


211 [227, 13291, 277]


In [14]:
%%time
P=Primes()
n=10**5
q=7
p=211
while p < 300: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

223 [631, 2137, 61]


227 [11719, 13487, 193]


229 [1201, 3023, 239]


233 [419, 2671, 449]


239 [2017, 28309, 89]


...

277 [5189, 2663, 127]


281 [359, 1831, 257]


283 [5059, 3229, 149]


293 [6277, 8089, 79]


307 [4219, 9227, 317]
CPU times: user 53min 21s, sys: 16.7 s, total: 53min 38s
Wall time: 1h 53min 9s


In [15]:
%%time
P=Primes()
n=10**5
q=7
p=301
while p < 500: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

307 [4219, 9227, 317]


311 [5527, 1487, 463]


313 [5351, 47629, 47]


317 [15271, 1663, 149]


331 [27701, 3547, 13]


...

479 [733, 6101, 2551]


487 [991, 2417, 1439]


491 [4861, 3307, 569]


499 [19319, 8387, 1621]


503 [4639, 3229, 811]
CPU times: user 8h 57min 57s, sys: 3min 33s, total: 9h 1min 30s
Wall time: 16h 9min 39s


In [16]:
%%time
P=Primes()
n=10**5
q=7
p=503
while p < 600: 
    p=P.next(p)
    f=fekete(q,p)
    g = reduced_fekete(f)
    print(p, triple(g,n))

509 [919, 4021, 2609]


521 [6547, 2843, 101]


523 [43579, 36653, 251]


541 [14683, 3643, 59]


547 [21011, 3079, 281]


...

577 [887, 17419, 409]


587 [3407, 28517, 643]


593 [18859, 73823, 433]


599 [32579, 34211, 193]


601 [7309, 26539, 547]
CPU times: user 15h 21min 14s, sys: 5min 20s, total: 15h 26min 35s
Wall time: 1d 7h 7min 26s


# Galois group of $f_{7p}$. 

First, we consider the case $p \equiv 1 \pmod{7}$. In this case, the discriminant of $f_{7p}$ is not a perfect square. To show that the Galois group of $f_{7p}$ is $(\Z/2)^n \rtimes S_n$ where $\deg(f_{3p})=2n$, we need to show that the Galois group of $f_{7p}$ contains a 2-cycle. The following codes find the smallest prime $q$ such that the reduction of $f_{7p}$ modulo $q$ produces a 2-cycle. 

In [19]:
%%time
P=Primes()
n=10**5
q=7
p=7
while p < 100: 
    p=P.next(p)
    if p % q ==1: 
        f = fekete(q,p)
        print(p, two_cycle(f,n))

29 691


43 3517


71 2113
CPU times: user 58.6 s, sys: 573 ms, total: 59.2 s
Wall time: 2min 9s


In [20]:
%%time
P=Primes()
n=10**5
q=7
p=71
while p < 300: 
    p=P.next(p)
    if p % q ==1: 
        f = fekete(q,p)
        print(p, two_cycle(f,n))

113 181


127 523


197 1753


211 2287


239 23917


281 23203
CPU times: user 5h 18min 45s, sys: 1min 46s, total: 5h 20min 31s
Wall time: 11h 15min 52s


In [21]:
%%time
P=Primes()
n=10**5
q=7
p=300
while p < 400: 
    p=P.next(p)
    if p % q ==1: 
        f = fekete(q,p)
        print(p, two_cycle(f,n))

337 11161


379 13009
CPU times: user 5h 21min 10s, sys: 1min 42s, total: 5h 22min 52s
Wall time: 11h 23min 32s


In [2]:
%%time
P=Primes()
n=10**5
q=7
p=400
while p < 450: 
    p=P.next(p)
    if p % q ==1: 
        f = fekete(q,p)
        print(p, two_cycle(f,n))

421 33479


449 31337
CPU times: user 23h 26min 10s, sys: 2min 58s, total: 23h 29min 9s
Wall time: 2d 9min 25s


In [4]:
%%time
P=Primes()
n=10**5
q=7
p=450
while p < 465: 
    p=P.next(p)
    if p % q ==1: 
        f = fekete(q,p)
        print(p, two_cycle(f,n))

463 12911
CPU times: user 4h 37min 8s, sys: 31.5 s, total: 4h 37min 40s
Wall time: 7h 34min 44s


In [7]:
%%time
P=Primes()
n=10**5
q=7
p=463
while p < 490: 
    p=P.next(p)
    if p % q ==1: 
        f = fekete(q,p)
        print(p, two_cycle(f,n))

491 8179
CPU times: user 3h 13min 53s, sys: 22.7 s, total: 3h 14min 15s
Wall time: 6h 36min 11s


We next consider the case $p \not \equiv 1 \pmod{7}$. In this case, the discriminant of $f_{7p}$ is a perfect square. To verify that it is maximal, we need to find a 2-4 cycle. 

In [8]:
%%time
P=Primes()
n=10**5
q=7
p=7
while p < 100: 
    p=P.next(p)
    if p % q !=1: 
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

11 761


13 1093


17 4751


19 137
23 19


31 3433


...

79 1667


83 2663


89 9551


97 2777


101 4157
CPU times: user 17min 23s, sys: 2.65 s, total: 17min 26s
Wall time: 35min 32s


In [9]:
%%time
P=Primes()
n=10**5
q=7
p=100
while p < 200: 
    p=P.next(p)
    if p % q !=1: 
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

101 4157


103 10487


107 9677


109 4933


131 10651


...

179 4957


181 10631


191 4127


193 79


199 701
CPU times: user 2h 50min 5s, sys: 16.5 s, total: 2h 50min 21s
Wall time: 3h 8min 56s


In [10]:
%%time
P=Primes()
n=10**5
q=7
p=200
while p < 300: 
    p=P.next(p)
    if p % q !=1: 
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

223 61


227 33937


229 34511


233 4231


241 2593


...

271 17851


277 15271


283 14821


293 18013


307 42727
CPU times: user 1d 2h 57min 6s, sys: 2min 36s, total: 1d 2h 59min 42s
Wall time: 1d 16h 59min 48s


In [11]:
%%time
P=Primes()
n=10**5
q=7
p=307
while p < 350: 
    p=P.next(p)
    if p % q !=1: 
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

311 829


313 1031


317 17881


331 2339


347 19231


349 77239


353 34849
CPU times: user 1d 6h 10min 20s, sys: 2min 48s, total: 1d 6h 13min 8s
Wall time: 2d 1h 7min 4s


In [12]:
%%time
P=Primes()
n=10**5
q=7
p=353
while p < 370: 
    p=P.next(p)
    if p % q !=1: 
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

359 19139


367 39079


373 6361
CPU times: user 13h 51min 13s, sys: 1min 19s, total: 13h 52min 32s
Wall time: 1d 2min 27s


In [13]:
%%time
P=Primes()
n=10**5
q=7
p=373
while p < 400: 
    p=P.next(p)
    if p % q !=1: 
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

383 55337


389 41659


397 7151


401 30403
CPU times: user 1d 11h 9min 25s, sys: 3min 51s, total: 1d 11h 13min 17s
Wall time: 2d 20h 57min 32s


In [14]:
%%time
P=Primes()
n=10**5
q=7
p=401
while p < 450: 
    p=P.next(p)
    if p % q !=1: 
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

409 12919


419 23431


431 14057


433 53633


439 31013


443 12541


457 20023
CPU times: user 2d 22h 35min 30s, sys: 11min 22s, total: 2d 22h 46min 53s
Wall time: 8d 16h 50min 25s


In [15]:
%%time
P=Primes()
n=10**5
q=7
p=457
while p < 500: 
    p=P.next(p)
    if p % q !=1: 
        f = fekete(q,p)
        print(p, search_two_four_cycle(f,n))

461 14407


467 48437


479 20117


487 20753


499 2531


503 5869
CPU times: user 2d 11h 19min 54s, sys: 9min 15s, total: 2d 11h 29min 9s
Wall time: 7d 13h 7min 21s
